# LSTM support

The following notebook contains a walkthrough of the support for the LSTM module

The notebook is organized in the following sections:
1. Icefabric API support
2. Icefabric CLI support

In [1]:
# Running imports
import json
import os
import threading
import zipfile
from pathlib import Path

import httpx

# Changes the current working dir to be the project root
current_working_dir = Path.cwd()
os.chdir(Path.cwd() / "../../")
print(
    f"Changed current working dir from {current_working_dir} to: {Path.cwd()}. This must run at the project root"
)

Changed current working dir from /home/daniel.cumpton/icefabric/examples/initial_parameters to: /home/daniel.cumpton/icefabric. This must run at the project root


In [2]:
# Starting the API locally
def run_api():
    """Starts the icefabric API locally"""
    !python -m app.main --catalog sql


threading.Thread(target=run_api).start()

INFO:     Will watch for changes in these directories: ['/home/daniel.cumpton/icefabric']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [3772608] using WatchFiles
INFO:     Started server process [3772671]
INFO:     Waiting for application startup.
Loading existing network graph from disk for: conus_hf
Loading existing network graph from disk for: ak_hf
Loading existing network graph from disk for: gl_hf
Loading existing network graph from disk for: hi_hf
Loading existing network graph from disk for: prvi_hf
INFO:     Application startup complete.
Starting subset for gages-01010000
Found origin flowpath: wb-4581
Tracking 170 total upstream segments
Subsetting network layer
Subsetting flowpaths layer
Subsetting nexus layer
Subsetting divides layer
Subsetting divide-attributes layer
INFO:     127.0.0.1:59456 - "GET /v1/modules/lstm/?identifier=01010000&domain=conus_hf&use_schaake=false HTTP/1.1" 200 OK


This will start the API on localhost:8000. This can be visited at: http://localhost:8000/docs 

![Icefabric API](../../docs/img/icefabric_api.png)

To test the SFT endpoint, we can use an example API call. This will return all config entries in JSON form. Each item in the output is the BMI Config contents for a specific catchment upstream of USGS 01010000

In [3]:
# Make the request
response = httpx.get(
    "http://0.0.0.0:8000/v1/modules/lstm/",
    params={
        "identifier": "01010000",  # the Gauge ID we're testing
        "domain": "conus_hf",  # The CONUS domain
        "use_schaake": "false",  # Specifying we're not using Schaake for the ice fraction setting
    },
    timeout=60.0,  # GLUE API requests can be slow depending on the network speed. Adding a 30s timeout to ensure requests go through
)

print(f"Status code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Status code: 200
[
  {
    "catchment": "cat-4564",
    "area_sqkm": 65.10285020700276,
    "basin_id": "gages-01010000",
    "basin_name": "",
    "elev_mean": 392.67774687853324,
    "inital_state": "zero",
    "lat": -70.13892643795896,
    "lon": 46.398022468189986,
    "slope_mean": 38.919279112997955,
    "timestep": "1 hour",
    "train_cfg_file": "",
    "verbose": "0"
  },
  {
    "catchment": "cat-4568",
    "area_sqkm": 7.097400207001865,
    "basin_id": "gages-01010000",
    "basin_name": "",
    "elev_mean": 366.7248617614768,
    "inital_state": "zero",
    "lat": -70.00487459822983,
    "lon": 46.507973117315515,
    "slope_mean": 43.05075248625828,
    "timestep": "1 hour",
    "train_cfg_file": "",
    "verbose": "0"
  },
  {
    "catchment": "cat-4575",
    "area_sqkm": 26.379449856000228,
    "basin_id": "gages-01010000",
    "basin_name": "",
    "elev_mean": 353.3614140779505,
    "inital_state": "zero",
    "lat": -69.88320459514527,
    "lon": 46.53046394628151,


### CLI

Now that we can return to the user all of the information / IPEs for a module, we can use the icefabric CLI to generate config files to disk for the user


In [4]:
Path("/tmp/lstm_output_test").mkdir(exist_ok=True, parents=True)

# Ensure the current working dir is where your `.pyiceberg.yaml` file is located
!uv run icefabric params --gauge 01010000 --nwm-module lstm --domain conus_hf --catalog sql --output /tmp/lstm_output_test

Loading existing network graph from disk for: conus_hf
Starting subset for gages-01010000
Found origin flowpath: wb-4581
Tracking 170 total upstream segments
Subsetting network layer
Subsetting flowpaths layer
Subsetting nexus layer
Subsetting divides layer
Subsetting divide-attributes layer
Creating a config file: 100%|██████████████████████████████████████████████████████████████████████████| 169/169 [00:00<00:00, 25429.14it/s]
Config files created successfully in the following folder: /tmp/lstm_output_test


Now that these are created, let's view the contents

In [5]:
with zipfile.ZipFile("/tmp/lstm_output_test/configs.zip", "r") as f:
    print("Files in archive:")
    for file_info in f.filelist:
        print(f"  {file_info.filename} ({file_info.file_size} bytes)")
    f.extractall("/tmp/lstm_output_test/configs")

Files in archive:
  cat-4564_bmi_config_lstm.txt (239 bytes)
  cat-4568_bmi_config_lstm.txt (237 bytes)
  cat-4575_bmi_config_lstm.txt (236 bytes)
  cat-4581_bmi_config_lstm.txt (237 bytes)
  cat-4758_bmi_config_lstm.txt (235 bytes)
  cat-4761_bmi_config_lstm.txt (234 bytes)
  cat-4820_bmi_config_lstm.txt (237 bytes)
  cat-4859_bmi_config_lstm.txt (239 bytes)
  cat-4862_bmi_config_lstm.txt (236 bytes)
  cat-4865_bmi_config_lstm.txt (236 bytes)
  cat-4866_bmi_config_lstm.txt (237 bytes)
  cat-4870_bmi_config_lstm.txt (237 bytes)
  cat-4874_bmi_config_lstm.txt (236 bytes)
  cat-4566_bmi_config_lstm.txt (237 bytes)
  cat-4570_bmi_config_lstm.txt (238 bytes)
  cat-4573_bmi_config_lstm.txt (236 bytes)
  cat-4574_bmi_config_lstm.txt (236 bytes)
  cat-4578_bmi_config_lstm.txt (235 bytes)
  cat-4579_bmi_config_lstm.txt (237 bytes)
  cat-4580_bmi_config_lstm.txt (237 bytes)
  cat-4749_bmi_config_lstm.txt (238 bytes)
  cat-4755_bmi_config_lstm.txt (234 bytes)
  cat-4764_bmi_config_lstm.txt (237 

Let's view one of their contents

In [7]:
files = Path("/tmp/lstm_output_test/configs").glob("*")
first_file = list(files)[0]
content = first_file.read_text()
print(content)

area_sqkm: 5.192999833499175
basin_id: gages-01010000
basin_name: 
elev_mean: 340.5621527962976
inital_state: zero
lat: -69.78632341528159
lon: 46.72490938929001
slope_mean: 55.08073454429075
timestep: 1 hour
train_cfg_file: 
verbose: 0


We can also view the metadata.json file that was created. This will contain additional information about the query parameters used to make these configs

In [8]:
metadata = Path("/tmp/lstm_output_test/configs/metadata.json")
content = json.loads(metadata.read_text())

print(content)

{'gauge_id': '01010000', 'domain': 'conus_hf', 'version': '2025.7.1', 'module': 'lstm', 'catalog_type': 'sql'}


Through either the API, or CLI, any modeler can create a BMI Config file for SFT that is compatible with NextGen. Now, let's clean up the `/tmp/` dir

In [ ]:
!rm -rf /tmp/lstm_output_test